In [21]:
import numpy as np
import pandas as pd
import librosa
import sklearn
import os
import pickle
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

In [22]:
# names = []
# Letter_names = ["M", "S", "B", "Y"]
# for letter in Letter_names:
#     for index in range(15):
#         names.append({'file':"{0}_open_({1}).wav".format(letter, index), 'label':"{0}".format(letter)})
# file_name = pd.DataFrame(names)
# print(file_name)

In [23]:
Names = os.listdir("AllRecords//Others")
Others_Names_df = pd.DataFrame(Names, columns={'Paths'})
Others_Names_df.insert(1, "Labels", len(Names) *[0])
print(Others_Names_df)

                        Paths  Labels
0    AuphonicRecording_16.wav       0
1    AuphonicRecording_17.wav       0
2    AuphonicRecording_18.wav       0
3    AuphonicRecording_19.wav       0
4    AuphonicRecording_20.wav       0
..                        ...     ...
169            r_open_(5).wav       0
170            r_open_(6).wav       0
171            r_open_(7).wav       0
172            r_open_(8).wav       0
173            r_open_(9).wav       0

[174 rows x 2 columns]


In [48]:
#Getting Team information
team_members = ['Mahmoud Hamdy','Sherif', 'yassmen', 'bassma']
iterator = 0
Team_df = pd.DataFrame()
labels = pd.DataFrame()
for member in team_members:
    iterator += 1
    Team = os.listdir("AllRecords/Team/{0}/Open_the_door".format(member))
    Team_df = Team_df.append(Team,ignore_index=True)
    #Team_df.insert(1, "Label", [iterator]*len(Team))
    labels = labels.append([iterator]*len(Team),ignore_index=True)
# Team_df.insert(1, "Label", labels)
# Data = {
#     "Paths": names,
#     "Labels": labels
# }
# Team_df = pd.DataFrame(Data)
labels = labels.rename(columns={0:"Labels"})
Team_df = Team_df.rename(columns={0:"Paths"})
# print(labels)
# print(Team_df)
Team_df = pd.concat([Team_df,labels], axis=1)
# print(Team_df)
all = Team_df.append(Others_Names_df, ignore_index=True)
print(all)

                            Paths  Labels
0    Mahmoud H Recording (17).wav       1
1    Mahmoud H Recording (18).wav       1
2    Mahmoud H Recording (19).wav       1
3    Mahmoud H Recording (20).wav       1
4    Mahmoud H Recording (21).wav       1
..                            ...     ...
321                r_open_(5).wav       0
322                r_open_(6).wav       0
323                r_open_(7).wav       0
324                r_open_(8).wav       0
325                r_open_(9).wav       0

[326 rows x 2 columns]


C:\Users\mahmo\AppData\Local\Temp\ipykernel_17512\2054682965.py:9: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  Team_df = Team_df.append(Team,ignore_index=True)
C:\Users\mahmo\AppData\Local\Temp\ipykernel_17512\2054682965.py:11: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  labels = labels.append([iterator]*len(Team),ignore_index=True)
C:\Users\mahmo\AppData\Local\Temp\ipykernel_17512\2054682965.py:24: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  all = Team_df.append(Others_Names_df, ignore_index=True)


In [49]:
def Extract_features(paths, whichone, dowhat = ''):
    # print(paths)
    # path = os.listdir("AllRecords/{0}/{1}/{2}".format(whichone, dowhat, paths["Paths"]))
    # try:
    path = os.path.join(os.path.abspath('AllRecords/{0}').format(whichone)+ ('\\') +str(paths['Paths']))
    X, sample_rate = librosa.load(path, res_type = "kaiser_fast")
    # except:
    #     print(paths["Paths"])   Voices_frequencies
    # print(path)
    # mfccs = np.mean(librosa.feature.mfcc(Voices_frequencies, sample_rate, n_mfcc = 40).T, axis = 0)
    # mel = np.mean(librosa.feature.melspectrogram(Voices_frequencies, sample_rate).T, axis = 0)
    # tonnetz = np.mean(librosa.feature.tonnetz(librosa.effects.harmonic(Voices_frequencies), sample_rate).T, axis = 0)
    # frequencies_in_stft = np.abs(librosa.stft(Voices_frequencies))
    # chroma = np.mean(librosa.feature.chroma_stft(frequencies_in_stft, sample_rate).T, axis = 0)
    # contrast = np.mean(librosa.feature.spectral_contrast(frequencies_in_stft, sample_rate).T, axis = 0)
    # label = paths['Labels']
    # return mfccs, mel, tonnetz, chroma, contrast, label
    # Generate Mel-frequency cepstral coefficients (MFCCs) from a time series 
    mfccs = np.mean(librosa.feature.mfcc(y=X, sr=sample_rate, n_mfcc=40).T,axis=0)

    # Generates a Short-time Fourier transform (STFT) to use in the chroma_stft
    stft = np.abs(librosa.stft(X))

    # Computes a chromagram from a waveform or power spectrogram.
    chroma = np.mean(librosa.feature.chroma_stft(S=stft, sr=sample_rate).T,axis=0)

    # Computes a mel-scaled spectrogram.
    mel = np.mean(librosa.feature.melspectrogram(X, sr=sample_rate).T,axis=0)

    # Computes spectral contrast
    contrast = np.mean(librosa.feature.spectral_contrast(S=stft, sr=sample_rate).T,axis=0)

    # Computes the tonal centroid features (tonnetz)
    tonnetz = np.mean(librosa.feature.tonnetz(y=librosa.effects.harmonic(X),
    sr=sample_rate).T,axis=0)
        
    
    # We add also the classes of each file as a label at the end
    return [mfccs, chroma, mel, contrast, tonnetz]
# def extract_features(files):
    
#     # Sets the name to be the path to where the file is in my computer
#     file_name = os.path.join(os.path.abspath('all')+'/'+str(files.file))

#     # Loads the audio file as a floating point time series and assigns the default sample rate
#     # Sample rate is set to 22050 by default
#     X, sample_rate = librosa.load(file_name, res_type='kaiser_fast') 

#     # Generate Mel-frequency cepstral coefficients (MFCCs) from a time series 
#     mfccs = np.mean(librosa.feature.mfcc(y=X, sr=sample_rate, n_mfcc=40).T,axis=0)

#     # Generates a Short-time Fourier transform (STFT) to use in the chroma_stft
#     stft = np.abs(librosa.stft(X))

#     # Computes a chromagram from a waveform or power spectrogram.
#     chroma = np.mean(librosa.feature.chroma_stft(S=stft, sr=sample_rate).T,axis=0)

#     # Computes a mel-scaled spectrogram.
#     mel = np.mean(librosa.feature.melspectrogram(X, sr=sample_rate).T,axis=0)

#     # Computes spectral contrast
#     contrast = np.mean(librosa.feature.spectral_contrast(S=stft, sr=sample_rate).T,axis=0)

#     # Computes the tonal centroid features (tonnetz)
#     tonnetz = np.mean(librosa.feature.tonnetz(y=librosa.effects.harmonic(X),
#     sr=sample_rate).T,axis=0)
        
    
#     # We add also the classes of each file as a label at the end
#     label = files.label

#     return mfccs, chroma, mel, contrast, tonnetz, label


In [50]:
mfccs = []
mel = []
tonnetz = []
chroma = []
contrast = []
label_enter = []
DataSet = pd.DataFrame()
# print(Team_df['Paths'])
DataSet = all.apply(Extract_features, whichone="AllTeams", axis=1)
# for i in range(len(Team_df)):
#     vmfcc, vmel, vtonnetz, vchroma, vcontrast, vlabel = Extract_features(Team_df[i], "Team", "Open_the_door")
#     mel.append(vmel)
#     tonnetz.append(vtonnetz)
#     chroma.append(vchroma)
#     contrast.append(vcontrast)
#     label_enter.append(vlabel)
#     mfccs.append(vmfcc)
# Data = {
#     "mfccs": mfccs,
#     "mel": mel,
#     "chroma": chroma,
#     "tonnetz": tonnetz,
#     "contrast": contrast,
#     "Labels": label_enter
# }
# DataSet = pd.DataFrame(Data)
dd = np.array(DataSet[0])
print(dd.shape)

c:\Python310\lib\site-packages\librosa\core\spectrum.py:222: UserWarning: n_fft=1024 is too small for input signal of length=569
  warnings.warn(
c:\Python310\lib\site-packages\librosa\core\spectrum.py:222: UserWarning: n_fft=1024 is too small for input signal of length=593
  warnings.warn(
c:\Python310\lib\site-packages\librosa\core\spectrum.py:222: UserWarning: n_fft=1024 is too small for input signal of length=297
  warnings.warn(
c:\Python310\lib\site-packages\librosa\core\spectrum.py:222: UserWarning: n_fft=1024 is too small for input signal of length=889
  warnings.warn(
c:\Python310\lib\site-packages\librosa\core\spectrum.py:222: UserWarning: n_fft=1024 is too small for input signal of length=445
  warnings.warn(
c:\Python310\lib\site-packages\librosa\core\spectrum.py:222: UserWarning: n_fft=1024 is too small for input signal of length=882
  warnings.warn(
c:\Python310\lib\site-packages\librosa\core\spectrum.py:222: UserWarning: n_fft=1024 is too small for input signal of length

(5,)


C:\Users\mahmo\AppData\Local\Temp\ipykernel_17512\3263792091.py:27: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  dd = np.array(DataSet[0])


In [51]:
print(DataSet[0])

[array([-4.7925879e+02,  1.2588013e+02, -1.8680016e+01,  2.8066908e+01,
       -3.7592990e+00,  9.1917505e+00, -9.6640539e+00,  4.2108774e+00,
       -1.7365740e+01,  1.7881201e+01, -1.0211505e+01,  8.9322004e+00,
       -2.5660553e+00,  6.6504383e-01,  2.2090255e-01,  1.2934023e+01,
       -1.0148947e+01,  7.4683805e+00, -4.2246504e+00, -3.7800226e-01,
       -2.6198735e+00,  1.6183909e+00, -5.1632667e+00, -3.7916157e+00,
       -2.6541629e+00,  3.6528273e+00,  2.6911230e+00, -8.9992732e-02,
       -1.0054581e+00,  3.9458923e+00,  5.2003753e-01,  1.8348803e-01,
        1.9236211e+00, -1.0041772e+00, -1.2911556e+00, -2.8943110e+00,
        1.3207041e+00,  1.3195422e+00,  4.5531881e-01, -1.7236742e+00],
      dtype=float32), array([0.53388834, 0.4819415 , 0.41172197, 0.44621053, 0.54601073,
       0.4835603 , 0.5545465 , 0.5410869 , 0.5337066 , 0.58438694,
       0.64982086, 0.62480193], dtype=float32), array([3.22894715e-02, 5.50554274e-03, 1.39400233e-02, 3.73665057e-02,
       2.4282

In [52]:
def features_append(features):
    feature = []
    for i in range(len(features)):
        feature.append(np.concatenate((features[i][0], features[i][1], features[i][2]
        , features[i][3],features[i][4])))
    return feature


In [53]:
x = features_append(DataSet)
# df = pd.DataFrame(x)
# print(df)

In [54]:
X_train, X_test, y_train, y_test = train_test_split(x, all['Labels'], test_size=0.2, random_state=0)

In [55]:
clf = RandomForestClassifier(random_state=0, n_estimators=40)
clf.fit(X_train,y_train.values.ravel())

RandomForestClassifier(n_estimators=40, random_state=0)

In [56]:
y_pred = clf.predict(X_test)

In [57]:
clf.predict(X_test)

array([4, 2, 0, 0, 0, 2, 0, 1, 4, 0, 0, 2, 1, 2, 0, 0, 0, 3, 2, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 2, 2, 0, 4, 2, 0, 1, 4, 0,
       1, 0, 0, 0, 0, 4, 1, 4, 2, 0, 3, 0, 0, 2, 2, 0, 0, 0, 2, 2, 0, 0],
      dtype=int64)

In [58]:
y_test

122    4
66     2
142    4
245    0
146    4
      ..
234    0
74     2
52     2
235    0
214    0
Name: Labels, Length: 66, dtype: int64

In [59]:
a = accuracy_score(y_pred, y_test)
print(a)

0.9242424242424242


In [60]:
clf.predict(x[:12])

array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], dtype=int64)

In [61]:
all['Labels'][:12]

0     1
1     1
2     1
3     1
4     1
5     1
6     1
7     1
8     1
9     1
10    1
11    1
Name: Labels, dtype: int64

In [68]:
newTest = os.listdir("AllRecords/Test/")
Test = pd.DataFrame(newTest, columns={"Paths"})
get_feat = Test.apply(Extract_features, whichone='Test', axis=1)
combine = features_append(get_feat)

c:\Python310\lib\site-packages\librosa\core\spectrum.py:222: UserWarning: n_fft=1024 is too small for input signal of length=986
  warnings.warn(
c:\Python310\lib\site-packages\librosa\core\spectrum.py:222: UserWarning: n_fft=1024 is too small for input signal of length=493
  warnings.warn(
c:\Python310\lib\site-packages\librosa\core\spectrum.py:222: UserWarning: n_fft=1024 is too small for input signal of length=869
  warnings.warn(
c:\Python310\lib\site-packages\librosa\core\spectrum.py:222: UserWarning: n_fft=1024 is too small for input signal of length=724
  warnings.warn(
c:\Python310\lib\site-packages\librosa\core\spectrum.py:222: UserWarning: n_fft=1024 is too small for input signal of length=362
  warnings.warn(
c:\Python310\lib\site-packages\librosa\core\spectrum.py:222: UserWarning: n_fft=1024 is too small for input signal of length=831
  warnings.warn(
c:\Python310\lib\site-packages\librosa\core\spectrum.py:222: UserWarning: n_fft=1024 is too small for input signal of length

In [69]:
clf.predict(combine)

array([4, 0, 0, 0, 0, 1, 2, 2, 3], dtype=int64)

In [70]:
pickle.dump(clf, open("Speech2-model.pkl", 'wb'))